In [49]:
import os
from chromadb.config import Settings


In [50]:
# Define the chroma settings
CHROMA_SETTINGS = Settings(
    chroma_db_impl = 'duckdb+parquet',
    persist_directory = 'db',
    anonymized_telemetry = False,
)

In [38]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, PDFMinerLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings 
from langchain.vectorstores import Chroma
import os
from constants import CHROMA_SETTINGS


In [6]:
persist_directory = 'db'

In [57]:
def main():
    for root, dirs, files in os.walk('docs'):
        for file in files:
            if file.endswith(".txt"):
                print(file)
                loader = TextLoader(os.path.join(root,file))
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    texts = text_splitter.split_documents(documents)
    
    # Create embeddings
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    # Create vector store
    db = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)
    
    db.persist()
    db=None
    

In [58]:
main()

subtitles.txt


d:\Python Hub\GenAI\Youtube Chatbot\youtube chatbot test\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [59]:
# Create embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [60]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

In [61]:
retriever = vectordb.as_retriever()

In [62]:
docs = retriever.get_relevant_documents("How much money did  raise?")

In [63]:
docs

[Document(page_content="and fall somewhere around15 to 18 lakhs for every 10 seconds andthis is where we saw the commercials ofdream 11 Nimbus Pepsi and so on and soforth so Sony's revenue is equal to thenumber of AD slots sold multiplied bythe amount paid per ad slot minus thecost of the broadcasting rights thenonce the expenses are factored in we'llget the profits generated by thebroadcasting partner and similar is theequation of the streaming partner alsoso initially when the league is not atall popular the", metadata={'source': 'docs\\subtitles.txt'}),
 Document(page_content="and fall somewhere around15 to 18 lakhs for every 10 seconds andthis is where we saw the commercials ofdream 11 Nimbus Pepsi and so on and soforth so Sony's revenue is equal to thenumber of AD slots sold multiplied bythe amount paid per ad slot minus thecost of the broadcasting rights thenonce the expenses are factored in we'llget the profits generated by thebroadcasting partner and similar is theequation of t

Model

In [65]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64
import textwrap
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain. vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA



In [66]:
checkpoint = "MBZUAI/LaMini-T5-738M"

In [67]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

d:\Python Hub\GenAI\Youtube Chatbot\youtube chatbot test\myenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PRITAM\.cache\huggingface\hub\models--MBZUAI--LaMini-T5-738M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [69]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    device_map="auto",
    torch_dtype=torch.float32,
    offload_folder="offload"

)

In [70]:
@st.cache_resource
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=model,
        tokenizer=tokenizer,
        max_length=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95

    )

    local_llm=HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [71]:
def qa_llm():
    llm=llm_pipeline()
    embeddings=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db=Chroma(persist_directory="db",embedding_function=embeddings)
    retriever=db.as_retriever()
    qa=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )

    return qa

In [ ]:
def process_answer(instruction):
    response=''
    instruction=instruction
    qa = qa_llm()
    generated_text=qa(instruction)
    answer=generated_text['result']
    return answer, generated_text


In [82]:
process_answer("how did ipl's genius business model make it a money machine for india?")

d:\Python Hub\GenAI\Youtube Chatbot\youtube chatbot test\myenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


("The answer is: The IPL's genius business model made it a money machine for India by allowing the league to attract a large number of players and generate a huge revenue stream through ticket sales, sponsorships, and merchandise sales.",
 {'query': "how did ipl's genius business model make it a money machine for india?",
  'result': "The answer is: The IPL's genius business model made it a money machine for India by allowing the league to attract a large number of players and generate a huge revenue stream through ticket sales, sponsorships, and merchandise sales.",
  'source_documents': [Document(page_content="rise to some of the richest playsin the world like Shaq and MichaelJordan therefore BCCI wanted to build asimilar product for Indian teams inIndia and this is how ladies andgentlemen the idea of the Indian PremierLeague came to life and little did theyknow that in the next 10 years thisVenture will go on to become one of thebiggest Sensations in sports history sothe question is

In [76]:
# def main():
#     st.title("Search Your PDF & ")
#     with st.expander("ABout the App"):
#         st.markdown(
        
#             """
#             This is a Generative AI powered Question and Answering app that responds to questions
#             about your PDF File.
#             """
#         )
#     question = st.text_area("Enter Your Question")
#     if st.button("Search"):
#         st.info("Your question: "+ question)
#         st.info("Your Answer")
#         answer,metadata - process_answer(question)
#         st.write(answer)
#         st.write(metadata)


In [77]:
main()

2024-05-28 02:46:56.394 
  command:

    streamlit run d:\Python Hub\GenAI\Youtube Chatbot\youtube chatbot test\myenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-05-28 02:46:56.444 Session state does not function when running a script without `streamlit run`
